<a href="https://colab.research.google.com/github/JSJeong-me/AutoGen/blob/main/ver049/AgentChat02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U "autogen-agentchat" "autogen-ext[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.2/234.2 kB 18.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Define a model client. You can use other model client that implements
# the `ChatCompletionClient` interface.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY",
)


# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()


---------- user ----------
What is the weather in New York?
---------- weather_agent ----------
[FunctionCall(id='call_u9h4Rzj918e1prrK8zgIIQeR', arguments='{"city":"New York"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_u9h4Rzj918e1prrK8zgIIQeR', is_error=False)]
---------- weather_agent ----------
The current weather in New York is 73 degrees and sunny.


In [ ]:
import asyncio
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)

    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant. Write all code in python. Reply only 'TERMINATE' if the task is done.",
        model_client=model_client,
    )

    code_executor = CodeExecutorAgent(
        name="code_executor",
        code_executor=LocalCommandLineCodeExecutor(work_dir="coding"),
    )

    # The termination condition is a combination of text termination and max message termination, either of which will cause the chat to terminate.
    termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(10)

    # The group chat will alternate between the assistant and the code executor.
    group_chat = RoundRobinGroupChat([assistant, code_executor], termination_condition=termination)

    # `run_stream` returns an async generator to stream the intermediate messages.
    stream = group_chat.run_stream(task="Write a python script to print 'Hello, world!'")
    # `Console` is a simple UI to display the stream.
    await Console(stream)

# asyncio.run(main())


In [4]:
# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- user ----------
Write a python script to print 'Hello, world!'
---------- assistant ----------
```python
print('Hello, world!')
```
---------- code_executor ----------
Hello, world!

---------- assistant ----------
TERMINATE


In [ ]:
# !pip install autogen-agentchat

In [ ]:
# !pip install autogen

In [ ]:
# from autogen.agentchat import AssistantAgent, UserProxyAgent, register_function

# llm_config = {
#     "config_list": [{"model": "gpt-4o", "api_key": "sk-xxx"}],
#     "seed": 42,
#     "temperature": 0,
# }

# tool_caller = AssistantAgent(
#     name="tool_caller",
#     system_message="You are a helpful assistant. You can call tools to help user.",
#     llm_config=llm_config,
#     max_consecutive_auto_reply=1, # Set to 1 so that we return to the application after each assistant reply as we are building a chatbot.
# )

# tool_executor = UserProxyAgent(
#     name="tool_executor",
#     human_input_mode="NEVER",
#     code_execution_config=False,
#     llm_config=False,
# )

# def get_weather(city: str) -> str:
#     return f"The weather in {city} is 72 degree and sunny."

# # Register the tool function to the tool caller and executor.
# register_function(get_weather, caller=tool_caller, executor=tool_executor)

# while True:
#     user_input = input("User: ")
#     if user_input == "exit":
#         break
#     chat_result = tool_executor.initiate_chat(
#         tool_caller,
#         message=user_input,
#         summary_method="reflection_with_llm", # To let the model reflect on the tool use, set to "last_msg" to return the tool call result directly.
#     )
#     print("Assistant:", chat_result.summary)
